In [5]:
# Import necessary libraries
import cv2  # For image processing
import pytesseract  # For Optical Character Recognition (OCR)
import numpy as np  # For array operations


In [6]:
# Set the path to the Tesseract-OCR executable
# Ensure you have Tesseract installed at this path
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"



In [7]:
# Load Haar Cascade classifier for license plate detection
plate_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_russian_plate_number.xml")



In [8]:
def preprocess_image(image):
    """
    Preprocess the input image to improve plate detection.
    - Convert the image to grayscale.
    - Equalize the histogram to handle lighting variations.
    """
    if image is None:
        print("Error: Image could not be loaded.")
        return None
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)
    return gray


In [9]:
def recognize_plate(plate_img):
    """
    Perform OCR on the cropped license plate image.
    - Use Tesseract with custom configurations for better accuracy.
    """
    plate_text = pytesseract.image_to_string(plate_img, config='--psm 8 --oem 3')
    return plate_text.strip()  # Remove any leading or trailing spaces


In [10]:
def detect_plate_in_image(image_path):
    """
    Detect license plates in an image and annotate the detected plates.
    - Draw rectangles around detected plates.
    - Use OCR to recognize and display the license plate text.
    """
    img = cv2.imread(image_path)
    if img is None:
        print("Error: Could not read image.")
        return

    # Preprocess the image
    gray = preprocess_image(img)
    if gray is None:
        return

    # Detect license plates in the image
    plates = plate_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(25, 25))

    for (x, y, w, h) in plates:
        # Draw a rectangle around the detected plate
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
        
        # Extract the plate region and recognize text
        plate_img = img[y:y + h, x:x + w]
        plate_text = recognize_plate(plate_img)
        
        # Annotate the image with the recognized text
        cv2.putText(img, f'Plate: {plate_text}', (x - 3, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the annotated image
    cv2.imshow('Number Plate Recognition', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [11]:
def detect_plate_in_video(video_path):
    """
    Detect license plates in a video and annotate frames in real time.
    - Draw rectangles around detected plates in each frame.
    - Use OCR to recognize and display license plate text.
    """
    video = cv2.VideoCapture(video_path)
    if not video.isOpened():
        print("Error: Could not open video.")
        return

    while True:
        ret, frame = video.read()
        if not ret:
            print("Error: Failed to read video frame.")
            break

        # Preprocess the current frame
        gray = preprocess_image(frame)
        if gray is None:
            continue

        # Detect license plates in the frame
        plates = plate_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(25, 25))

        for (x, y, w, h) in plates:
            # Draw a rectangle around the detected plate
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            
            # Extract the plate region and recognize text
            plate_img = frame[y:y + h, x:x + w]
            plate_text = recognize_plate(plate_img)
            
            # Annotate the frame with the recognized text
            cv2.putText(frame, f'Plate: {plate_text}', (x - 3, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Display the annotated frame
        cv2.imshow('Video Plate Recognition', frame)

        # Press 'q' to stop the video
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    video.release()
    cv2.destroyAllWindows()


In [15]:
# Run the detection on an image
detect_plate_in_image('images/Cars415.png')

# Run the detection on a video
detect_plate_in_video('vid.mp4')


KeyboardInterrupt: 

In [ ]:
import cv2
import pytesseract
import numpy as np
import re
from pathlib import Path

# Set Tesseract path
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

class LicensePlateDetector:
    def __init__(self):
        self.plate_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + "haarcascade_russian_plate_number.xml"
        )
        if self.plate_cascade.empty():
            raise ValueError("Error: Haar cascade file could not be loaded.")

    def preprocess_image(self, image):
        """Enhanced image preprocessing"""
        if image is None:
            raise ValueError("Error: Image could not be loaded.")

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        blur = cv2.bilateralFilter(gray, 11, 17, 17)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        enhanced = clahe.apply(blur)
        return enhanced

    def preprocess_plate(self, plate_img):
        """Enhance extracted plate region"""
        min_width = 300
        if plate_img.shape[1] < min_width:
            aspect_ratio = plate_img.shape[0] / plate_img.shape[1]
            new_width = min_width
            new_height = int(aspect_ratio * new_width)
            plate_img = cv2.resize(plate_img, (new_width, new_height))

        if len(plate_img.shape) == 3:
            gray = cv2.cvtColor(plate_img, cv2.COLOR_BGR2GRAY)
        else:
            gray = plate_img

        thresh = cv2.threshold(
            gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU
        )[1]
        kernel = np.ones((3, 3), np.uint8)
        cleaned = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
        cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_OPEN, kernel)

        return cleaned

    def validate_plate_text(self, text):
        """Clean and validate plate text"""
        text = re.sub(r"[^A-Z0-9]", "", text.upper())
        if len(text) < 4:
            return ""
        text = text.replace("O", "0").replace("I", "1").replace("S", "5")
        return text

    def recognize_plate(self, plate_img):
        """Enhanced OCR with custom configuration"""
        processed_plate = self.preprocess_plate(plate_img)
        custom_config = r"--oem 3 --psm 8 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"

        try:
            plate_text = pytesseract.image_to_string(processed_plate, config=custom_config)
            cleaned_text = self.validate_plate_text(plate_text)
            return cleaned_text

        except Exception as e:
            print(f"OCR Error: {str(e)}")
            return ""

    def detect_plate_in_image(self, image_path):
        """Process single image"""
        if not Path(image_path).is_file():
            print(f"Error: Image file not found at {image_path}")
            return

        img = cv2.imread(image_path)
        if img is None:
            print("Error: Could not read image.")
            return

        processed = self.preprocess_image(img)

        plates = self.plate_cascade.detectMultiScale(
            processed, scaleFactor=1.1, minNeighbors=5, minSize=(80, 20), maxSize=(300, 100)
        )

        for (x, y, w, h) in plates:
            padding = 5
            x = max(0, x - padding)
            y = max(0, y - padding)
            w = min(img.shape[1] - x, w + 2 * padding)
            h = min(img.shape[0] - y, h + 2 * padding)

            plate_img = img[y : y + h, x : x + w]
            plate_text = self.recognize_plate(plate_img)

            if plate_text:
                cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(
                    img, plate_text, (x, y - 10), font, 0.7, (0, 255, 0), 2
                )

        cv2.imshow("Number Plate Recognition", img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    def detect_plate_in_video(self, video_path):
        """Process video stream"""
        if not Path(video_path).is_file():
            print(f"Error: Video file not found at {video_path}")
            return

        video = cv2.VideoCapture(video_path)
        if not video.isOpened():
            print("Error: Could not open video.")
            return

        while True:
            ret, frame = video.read()
            if not ret:
                break

            processed = self.preprocess_image(frame)

            plates = self.plate_cascade.detectMultiScale(
                processed, scaleFactor=1.1, minNeighbors=5, minSize=(80, 20), maxSize=(300, 100)
            )

            for (x, y, w, h) in plates:
                plate_img = frame[y : y + h, x : x + w]
                plate_text = self.recognize_plate(plate_img)

                if plate_text:
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(
                        frame,
                        plate_text,
                        (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.7,
                        (0, 255, 0),
                        2,
                    )

            cv2.imshow("Video Plate Recognition", frame)
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break

        video.release()
        cv2.destroyAllWindows()


# Example usage
if __name__ == "__main__":
    detector = LicensePlateDetector()

    # Process image
    detector.detect_plate_in_image("images/Cars426.png")

    # Process video
    detector.detect_plate_in_video("vid.mp4")




KeyboardInterrupt: 